In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib notebook

# Scikit-learn Soup to Nuts: <br> Developing a Machine-Learning Workflow

In this lecture we will discuss the tools and steps necessary to build a successful machine-learning model.

<br>
<center> Adam A Miller  
CIERA/Northwestern & Adler Planetarium  
(c) 2017 Nov 2</center>

### Machine Learning

&nbsp;&nbsp;&nbsp;&nbsp; fundamentally concerned with the problem of classification   
&nbsp;&nbsp;&nbsp;&nbsp; *particularly in regime of large dimensional data sets*  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (methods can be extended to regression)  
  
&nbsp;&nbsp;&nbsp;&nbsp; (glorified) Pattern Matching  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *a slight over-simplification* 

In other words, be careful about over-interpreting the "learning"...

<center>
<img src="images/DL_vs_humans.png" width=850cm>
</center>

<center>
<font size="1"> *credit*: this image is everywhere on the web, and I cannot track down original source, [this blog](https://devblogs.nvidia.com/parallelforall/mocha-jl-deep-learning-julia/) uses it without attribution </font>
</center>

### Terminology

&nbsp;&nbsp;&nbsp;&nbsp; **Features**   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; measured properties of objects in the data set  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; can be numerical or categorical (e.g., red vs. blue)

&nbsp;&nbsp;&nbsp;&nbsp; **Labels**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; target classification or regression variable (to be predicted)

Standard ([supervised](https://en.wikipedia.org/wiki/Supervised_learning)) ML goal: 
  1. **Train** — Develop a mapping between *features* and *labels*

  2. **Test** — Evaluate model on non-training labeled data

  3. **Predict** — Apply model to sources with unknown labels

Today I will not discuss [unsupervised machine learning](https://en.wikipedia.org/wiki/Unsupervised_learning). Primarily because we do not have time, but also because I have not seen a single useful application of these techniques in my own science.

In brief, unsupervised learning ignores any labels that may be available and instead attempts to cluster sources based on their similarity in the multidimensional feature space. However, once the clusters have been identified there is no mathematical method for measuring the quality of the clusters (and hence my feelings that these methods aren't that useful).

<center>
<img src="images/ML_summary.png" width=850cm>
</center>

### Question 1

Why is the step with the test data necessary?

[Take a few min to dicuss with your partner]

With this simple picture in mind, let's get started.

Our tool for today is `python`'s [scikit-learn](http://scikit-learn.org/stable/).

`scikit-learn` is amazing! It includes everything needed to construct the ML workflow, and has excellent documentation. 

With only 4 (!) lines of code, we can build a ML model with `scitkit-learn`.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
iris = load_iris()
rf_clf = RandomForestClassifier().fit(iris.data, iris.target)

## Bang

Just like that - you're done. 

Now you can all go home.

As a very important aside - allow me a moment on my soapbox to advise caution regarding the simplicity of `scikit-learn`: the package is so user friendly, and documentation so good, that it is not just easy to build a model, but it is also incredibly easy to become over confident in the model. Generally speaking, ML models are highly subject to noise and training-set biases and the simplicity of `scikit-learn` can consistently lead to a few lines of code that appear to produce outstanding results.

This is the first (but it will not be the last) time that I will implore you to **worry about the data**

On to building a full pipeline...

## 1. Data Preparation

As ML is a data-driven method, the first, and arguably most important step is to curate the data.

  1. Query, observe, simulate, etc.   
&nbsp;(i.e. collect the observations)
  2. Select features to be used in the model
  3. Determine training set "ground truth" (i.e. *labels*)

Beyond these initial considerations, additional setps to consider include:  

&nbsp;&nbsp; 4. Convert categorical features  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; e.g., male, female, male, male $\rightarrow$ [0, 1, 0, 0]  
&nbsp;&nbsp; 5. [Impute](https://en.wikipedia.org/wiki/Imputation_(statistics) (or discard) missing data  
&nbsp;&nbsp; 6. Feature normalization  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; typically only necessary for certain ML models  
&nbsp;&nbsp; 7. Visualize the data   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a critical step for all data-science applications  

and of course, don't forget...

# Worry About the Data

Today we will work with the famous [iris flower data set](https://en.wikipedia.org/wiki/Iris_flower_data_set), which is small and understandable, but as a result avoids many of the trappings of dealing with real world data. 

There are 3 iris classes: setosa, virginica, and versicolor.

For each flower, 4 features have been measured: petal length, petal width, sepal length, and sepal width.

We will use [`seaborn`](https://seaborn.pydata.org) to visualize the data (but all subsequent work will be in `scikit-learn`).

In [ ]:
xkcd_colors = ["windows blue", "amber", "slate"]

def infer_cmap(color):
    xkcd_colors = ["windows blue", "amber", "slate"]
    hues = sns.xkcd_palette(xkcd_colors)
    if color == hues[0]:
        return sns.light_palette(hues[0], as_cmap=True)
    elif color == hues[1]:
        return sns.light_palette(hues[1], as_cmap=True)
    elif color == hues[2]:
        return sns.light_palette(hues[2], as_cmap=True)

def kde_color_plot(x, y, **kwargs):
    cmap = infer_cmap(kwargs['color'])
    ax = sns.kdeplot(x, y, shade=True, shade_lowest=False, cmap=cmap, **kwargs)
    return ax

In [ ]:
iris_df = sns.load_dataset("iris")
g = sns.PairGrid(iris_df, hue='species', 
                 vars=['sepal_length','sepal_width',
                       'petal_length','petal_width'], 
                 palette=sns.xkcd_palette(xkcd_colors), 
                 diag_sharey=False)
g = g.map_upper(plt.scatter, alpha=0.7)
g = g.map_lower(kde_color_plot)
g = g.map_diag(sns.kdeplot, shade=True)

In brief, these corner plots show that the data are fairly well separated, though there is some overlap between the virginica and versicolor species.

## 2. Feature Engineering

[This step may need to be repeated]

Add new features (if necessary)  
&nbsp;&nbsp;&nbsp;&nbsp; Utilize domain knowledge to create/compute new features    
&nbsp;&nbsp;&nbsp;&nbsp; e.g., sepal_length/petal_length may be more informative

Remove noisy/uniformative features (if necessary)  
&nbsp;&nbsp;&nbsp;&nbsp; [Feature importance can be measured](http://scikit-learn.org/stable/modules/ensemble.html#feature-importance-evaluation) via Random Forest  
&nbsp;&nbsp;&nbsp;&nbsp; [Forward/backward feature selection](https://www.cs.cmu.edu/~kdeng/thesis/feature.pdf) can thin feature set

In this case we have only 4 features, so we will proceed under the assumption that the feature set need not be thinned.

## 3. Model Selection

[This step may need to be repeated]

Following data organization and feature engineering, the practitioner must then select an ML algorithm.

Every problem/data set is different. Best practices often include trying multiple algorithms to determine which is best.

After lots of experience it is possible to develop some intuition for which algorithms will work best in which regimes.

But remember - ultimately we are working with black boxes. Intuition can easily lead you astray in this case...

We will adopt the [$k$-Nearest Neighbors](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) ($k$NN) algorithm for today's problem. The primary reason is that it is very easy to understand:

Final classifications are determined by identifying the $k$, a user-selected number, nearest neighbors in the training set to the source being classified. Euclidean distances are typically used to determine the separation between sources, though other metrics are also possible.

$k$NN is an algorithm that may require feature normalization (discussed above). 

Imagine, for example, a two feature model where feature $x_1$ is gaussian distributed with mean 0 and standard deviation 10 $[x_1 \sim \mathcal{N}(0, 100)]$, compared to feature $x_2 \sim \mathcal{N}(0,0.01)$. In this case, the classifications will be entirely decided by $x_1$ as the typical $\Delta x_1$ will be orders of magnitude larger than $\Delta x_2$. 

Of course, if $x_1$ is significantly more important than $x_2$ then maybe this is okay.

`scikit-learn` makes $k$NN easy with the [`KNeighborsClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) class from the [`neighbors`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.neighbors) subpackage.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=11)

You may have noticed that I set $k = 11$. This should worry you - why 11 neighbors and not 7? or the default 5? or 121?

We will answer that now...

The real answer for why I set $k = 11$ is that today I got a [slurpie](https://en.wikipedia.org/wiki/Slurpee) and it tasted good.

## 4. Model Evaluation

[This step may need to be repeated]

With model in hand, we now need to evaluate its performance.

What is the best metric for evaluating the selected model?

There are many metrics we can use to evalute a model's performance, and we will cover a few of those now.

Before we evaluate the model, we need to split the data into a training and test set (described above). We can easily do this using [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from the [`model_selection`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection) `scikit-learn` subpackage.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, 
                                                    iris.target, 
                                                    test_size = 0.3, 
                                                    random_state = 23)

Why random_state = 23? Because we are in Chicago, and Michael Jordan was on the Bulls, and Michael Jordan is the best basketball player ever.

At this stage - we set the test set aside, and ignore it completely until we have fully optimized our model.

Applying the model to these data before finalizing the model is SNOOPING - don't do it.

### Terminology

&nbsp;&nbsp;&nbsp;&nbsp; **True Positive** (TP)   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; + classified as +  

&nbsp;&nbsp;&nbsp;&nbsp; **False Positive** (FP)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - classified as +  

&nbsp;&nbsp;&nbsp;&nbsp; **True Negative** (TN)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - classified as -  

&nbsp;&nbsp;&nbsp;&nbsp; **False Negative** (FN)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; + classified as - 

### Terminology - Most metrics are defined by [TP, FP, TN, and FN](https://en.wikipedia.org/wiki/Sensitivity_and_specificity):

&nbsp;&nbsp;&nbsp;&nbsp; **Accuracy**    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (TP + TN)/(TP + TN + FP + FN)  
&nbsp;&nbsp;&nbsp;&nbsp; **True Positive Rate** (aka sensitivity, recall, etc)   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; TP/(TP + FN)  
&nbsp;&nbsp;&nbsp;&nbsp; **False Positive Rate**   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; FP/(TN + FP)  
&nbsp;&nbsp;&nbsp;&nbsp; **True Negative Rate** (aka specificity)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; TN/(TN + FP)  
&nbsp;&nbsp;&nbsp;&nbsp; **Precision**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; TP/(TP + FP)  

and many, many more...

Another extremely useful tool is the [Receiver Operating Characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) curve. For $k$NN it is not possible to determine the ROC curve, as the ROC curve is determined by measuring the TPR vs. FPR as a function of varying classification decision thresholds. Models that produce probabilistic classifications can be used to create ROC curves.

The ROC curve is extremely useful for setting decision thresholds in cases where a desired TPR or FPR is known a priori (e.g., when to trigger human review of credit card transactions in possible cases of fraud). When "follow-up" resources are limited setting these thresholds helps to optimize performance.

Finally, the [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) is exceptionally useful for identifying classes that are being misclassified:

<table>
  <tr>
  <td> </td><td> </td><td colspan="2">True Class</td>
  </tr>
  <tr>
  <td> </td><td> </td> <td> + </td><td> - </td> 
  </tr>
  <tr>
  <td> predicted </td><td> + </td> <td> TP </td><td> FN </td> 
  </tr>
  <tr>
  <td> class </td><td> - </td> <td> FP </td><td> TN </td> 
  </tr>
</table>


As we cannot touch the test set, how do we evaluate the model performance?

[Cross validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics). In brief, we will further split the training set, use some of it to define the mapping between features and labels, and then evaluate the quality of that mapping using the sources that were withheld from training.

There are many flavors of CV, but $k$-fold CV is most common. In $k$-fold CV, the training set is split into $k$ partitions. Iteratively, each partion is withheld, the model is trained, and predictions are made on the withheld partition. With predictions for every source in hand, we can compare the predictions to the known labels.

Cross validation is simple with `scikit-learn` using the `model_selection` subpackage. We will focus on [`cross_val_predict`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) which returns predictions for every source in the training set.

In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_preds = cross_val_predict(knn_clf, X_train, y_train, cv = 10)

The super useful [`metrics`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) subpackage allows us to evaluate the model. 

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

print("kNN CV acc = {:.4f}".format(accuracy_score(y_train, 
                                                  y_train_preds)))

We can also use `scikit-learn` to make a confusion matrix. 

A nice looking confusion matrix requires [more code than fits on a slide](http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py). I'll hide that and instead just show the results.

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

#     print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cmap = sns.cubehelix_palette(8, start=.5, rot=-.75, as_cmap=True)
cm = confusion_matrix(y_train, y_train_preds)
with sns.axes_style("white"):
    plot_confusion_matrix(cm, iris.target_names, 
                          normalize = True, 
                          cmap = cmap)

## 5. Model Optimization

[This step may need to be repeated]

Previously, we set $k = 11$ for the $k$NN model, but we (rightly) asked, what is so special about 11?

Now we should optimize the model tuning parameters.

The tried and true method in this regard is brute force: an exhaustive grid search across all relevant tuning parameters.

In cases with many parameters (or extremely large data sets) a [randomized parameter search may be more pragmatic](http://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py).

Unfortunately, there is no substitute for patience. 

It is virtually never the case that some objective function can be optimized to determine the optimal tuning parameters.

Using the `model_selection` package, we can use the [`GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) class to perform the exhaustive search.

In this case, the relevant parameters are $k$ and $p$, the order of the [Minkowski distance](https://en.wikipedia.org/wiki/Minkowski_distance) ($p = 2$ is equivalent to Euclidean distance).

In [ ]:
from sklearn.model_selection import GridSearchCV
opt_tune = GridSearchCV(knn_clf, {'n_neighbors': [1, 3, 5, 10, 30, 50], 
                                  'p': [1, 2, 3]}, cv = 10)
opt_tune.fit(X_train, y_train)
opt_k = opt_tune.best_params_['n_neighbors']
opt_p = opt_tune.best_params_['p']
print("Opt model has k = {:d} and p = {:d}".format(opt_k, opt_p))


Furthermore, it is useful to understand how the performance changes as a function of the parameters in the search.

In [ ]:
k_grid = np.unique(opt_tune.cv_results_['param_n_neighbors'])
p_grid = np.unique(opt_tune.cv_results_['param_p'])
K, P = np.meshgrid(k_grid, p_grid)

score_grid = np.empty(np.shape(K))
for params, acc in zip(opt_tune.cv_results_['params'], 
                       opt_tune.cv_results_['mean_test_score']):

    this_source = np.where((K == params['n_neighbors']) & (P == params['p']))
    score_grid[this_source] = acc

In [ ]:
with sns.axes_style('white'):
    fig, ax = plt.subplots()
    im = ax.imshow(score_grid, origin = 'lower_left', 
                   cmap = cmap)

    thresh = 0.92
    for i, j in itertools.product(range(score_grid.shape[0]), 
                                  range(score_grid.shape[1])):
        ax.text(j, i, format(score_grid[i, j], '.4f'),
                 horizontalalignment="center",
                 color="w" if score_grid[i, j] > thresh else "k")
    
    ax.set_xticks(np.arange(len(k_grid)))
    ax.set_xticklabels(k_grid)
    ax.set_yticks(np.arange(len(p_grid)))
    ax.set_yticklabels(p_grid)
    ax.set_xlabel('k', fontsize = 14)
    ax.set_ylabel('p', fontsize = 14)
    cb = plt.colorbar(im)

In this case we see that a range of different parameter choices provide the optimal results. Moving forward we will make predictions using models with $k = 10$ and $p = 2$.

## 6. Prediction

The final step, now that we have fully specified and trained our model, is to make predictions.

Apply the model to the test set $\rightarrow$ estimate the generalization error (i.e. how does the model perform on new data?).

The test-set generalization error typically overestimates the model performance. There are several reasons why this might be the case, but the easiest to understand is training set bias. 

Every source that has a label is labeled "for a reason" (typically because someone, somewhere decided to devote resources towards labeling). This selection process is rarely random, meaning the training set is biased relative to the population. These biases, even if small, will be propagated through the model, but not identified via the test set, which (in most cases) comes from the same super set as the training set. 

As always - **worry about the data**

Pulling back the curtain on the test set is a point of no return. 

At this stage, it's possible (and encouraged if necessary) to go back and adjust the work in section 2 (feature engineering), 3 (model selection), 4 (model evaluation) and 5 (model optimization). 

Cycling through these procedures multiple times is typically needed prior to evaluating the model with the test set.

We, however, will proceed with our simple model:

In [ ]:
knn_clf_final = KNeighborsClassifier(n_neighbors=3, p = 2)
knn_clf_final.fit(X_train, y_train)

test_preds = knn_clf_final.predict(X_test)

gen_error = 1 - accuracy_score(y_test, test_preds)

print("The kNN test-set acc = {:.4f}".format(1 - gen_error))

Interestingly, the test-set accuracy is $\approx$ the CV accuracy.

Thus, we should not expect this model to classify newly observed Iris flowers with an accuracy higher than 97%.

Of course, the most interesting part of any ML model is applying it to unlabeled data.

Unfortunately, we do not have unlabeled Iris data.

The steps described above lay out the path to build an end-to-end, production ready machine learning model. Using this approach, it should be possible for you to construct and train models for prediction within your own disciplines.

# The End

### Reading List

The standard for learning about Machine Learning is [*The Elements of Machine Learning*](https://web.stanford.edu/~hastie/Papers/ESLII.pdf) by Hastie, Tibshirani, & Friedman.

[`scikit-learn`](http://scikit-learn.org/stable/) is filled with examples and (brief) tutorials that cover a great deal of material beyond what was discussed today.

Finally, [*Real World Machine Learning*](https://www.manning.com/books/real-world-machine-learning) by Brink, Richards, and Fetherolf is a great, approachable text for those beginning with machine learning. Much of this lecture was inspired by principles from this book. *Full disclosure* - this book was written by my friends, but if you can't shill for your friends, then who can you shill for?

### Reading List 2

At present, the most exciting tools in machine learning are related to neural nets and "deep learning." There are 2 dominant libraries for these methods:

[`TensorFlow`](https://www.tensorflow.org)

[`Theano`](https://github.com/Theano/Theano)

Both of these libraries can be run in Python using [`Keras`](https://keras.io), a great place to start if you've never used these packages before.

There's a LOT written about these libraries and different forms of deep learning, but if you're looking for a place to start [*Deep Learning in Python*](https://www.manning.com/books/deep-learning-with-python) by Francois Chollet (author of `Keras`) would be a good place to start.

Additionally I'll note that some researchers have put together a package that in principle runs through all the steps I described above: [`auto-sklearn`](http://automl.github.io/auto-sklearn/stable/). While this package looks interesting and powerful, I would not recommend diving in here, without going through the above steps a few times. *Note - there is no substitution for domain knowledge*